# **Notebook Setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.1 MB/s eta 0:00:00


# **Imports**

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

# **Model and Tokenizer**

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

# **Data Loading**

In [5]:
drive_path = 'drive/MyDrive/MetalliA/'# dataset/ models/

In [6]:
def parse_lyrics(root, song)-> str:
  '''
    Removes title and empty rows from song
    Returns lyrics without linebreaks
  '''
  if song.__contains__('.txt'):
    with open(os.path.join(root, song), "r") as file:
      lyrics = file.read()
      lyrics = [line+' <|endofverse|>' for line in lyrics.split('\n') if len(line) > 0]
      lyrics = ' '.join(lyrics)
  return lyrics

**Data example for clarification**

root: drive/MyDrive/MetalliA/dataset/5-black_album

song: dont_tread_on_me.txt

In [7]:
data_path = drive_path + 'dataset/'

df = pd.DataFrame(columns=['album', 'song_name', 'lyrics'])

for root, dirs, files in os.walk(data_path):
  for song in files:
    #extract album name from path
    album = root.split('/')[-1]
    #remove album number from name
    album = album.split('-')[-1]

    #parse song name
    song_name = song.split('.')[0]
    song_name = ' '.join(song_name.split('_'))

    lyrics = parse_lyrics(root, song)

    row = {'album': album, 'song_name': song_name, 'lyrics': lyrics}
    df = df.append(row, ignore_index=True)


<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-7-60a2cc981eb9>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [8]:
print("total of songs: {}".format(len(df.index)))

total of songs: 106


In [9]:
df.head(10)

,album,song_name,lyrics
0,ride_the_lightning,ride the lightning,"""Ride The Lightning"" <|endofverse|> Guilty as ..."
1,ride_the_lightning,escape,"""Escape"" <|endofverse|> Feel no pain, but my l..."
2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno..."
3,ride_the_lightning,creeping death,"""Creeping Death"" <|endofverse|> Slaves <|endof..."
4,ride_the_lightning,fight fire with fire,"""Fight Fire With Fire"" <|endofverse|> Do unto ..."
5,ride_the_lightning,for whom the bell tolls,"""For Whom The Bell Tolls"" <|endofverse|> Make ..."
6,ride_the_lightning,fade to black,"""Fade To Black"" <|endofverse|> Life, it seems,..."
7,and_justice_for_all,shortest straw,"""The Shortest Straw"" <|endofverse|> Suspicion ..."
8,and_justice_for_all,the frayed ends of sanity,"""The Frayed Ends Of Sanity"" <|endofverse|> Nev..."
9,and_justice_for_all,eye of the beholder,"""Eye Of The Beholder"" <|endofverse|> Do you se..."


In [10]:
lir = list(df['lyrics'])
song_length = [len(s.split(' ')) for s in lir]
print(song_length)

[238, 270, 236, 287, 179, 191, 152, 384, 235, 413, 253, 358, 279, 206, 246, 312, 134, 229, 258, 266, 248, 207, 190, 441, 409, 354, 199, 138, 312, 217, 217, 244, 283, 213, 243, 276, 262, 347, 234, 305, 314, 280, 324, 260, 229, 170, 216, 193, 361, 285, 376, 210, 286, 346, 357, 327, 241, 213, 302, 368, 368, 161, 308, 499, 237, 272, 509, 183, 501, 275, 404, 599, 291, 310, 326, 322, 224, 481, 272, 348, 389, 371, 256, 379, 244, 258, 312, 244, 212, 263, 246, 260, 325, 335, 279, 433, 450, 254, 471, 264, 235, 249, 298, 419, 326, 332]


We split the data into train and test set.

In [11]:
test_set = df.sample(n = 10)
train_set = df.loc[~df.index.isin(test_set.index)]
print('train set size: {}'.format(len(train_set.index)))
print('test set size: {}'.format(len(test_set.index)))

train set size: 96
test set size: 10


In [12]:
test_set = test_set.reset_index()
train_set = train_set.reset_index()

In the test set, we remove the last part of the songs in order to measure the accuracy of our trained model

In [13]:
test_set['true_lyrics'] = test_set['lyrics'].str.split().str[-20:].apply(' '.join)
test_set['lyrics'] = test_set['lyrics'].str.split().str[:-20].apply(' '.join)

In [14]:
test_set.head(10)

,index,album,song_name,lyrics,true_lyrics
0,94,reload,slither,"""Slither"" <|endofverse|> Don't go looking for ...",here before? <|endofverse|> Have your heroes d...
1,83,black_album,the unforgiven,"""The Unforgiven"" <|endofverse|> New blood join...",you. <|endofverse|> So I dub thee unforgiven. ...
2,100,reload,better than you,"""Better Than You"" <|endofverse|> I look at you...","on and on <|endofverse|> Forever on and on, on..."
3,2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno...",<|endofverse|> Can't hear my call <|endofverse...
4,37,hardwired_to_self_destruct,atlas rise,"""Atlas, Rise!"" <|endofverse|> Bitterness and b...",Bound by the world all alone <|endofverse|> Cr...
5,74,death_magnetic,all nightmare long,"""All Nightmare Long"" <|endofverse|> Luck. Runs...",Luck. Runs. Out. <|endofverse|> You crawl back...
6,98,reload,fixxxer,"""Fixxxer"" <|endofverse|> Dolls of voodoo all s...","<|endofverse|> No more, no more pins in me <|e..."
7,84,black_album,through the never,"""Through The Never"" <|endofverse|> All that is...",<|endofverse|> Turning <|endofverse|> Who we a...
8,102,reload,carpe diem baby,"""Carpe Diem Baby"" <|endofverse|> Hit dirt <|en...",suck the day <|endofverse|> Come make me miss ...
9,76,death_magnetic,the day that never comes,"""The Day That Never Comes"" <|endofverse|> Born...",I swear <|endofverse|> The sun will shine <|en...


Create the dataset acording to GPT-2 expected format. 
Song max lenth is limited to 1024 tokens since GPT-2 performance gets worse from that length on.

We have to wrap it up into a class that inherits from torch Dataset so we can enumerate and loop over it.

In [15]:
class Lyrics(Dataset):
  def __init__(self, data, max_len = 1024):
    self.tokenizer = tokenizer
    self.lyrics = []

    for row in data['lyrics']:
      self.lyrics.append(
          torch.tensor(
            tokenizer.encode(f"<|[lyrics]|>{row[:max_len]}<|endoftext|>")
          )
      )
    
    self.count = len(self.lyrics)

  def __len__(self):
    return self.count

  def __getitem__(self, item):
    return self.lyrics[item]

In [16]:
tr_dataset = Lyrics(train_set)

# **Model Training**

First we define the hyper-parameters for our model

In [17]:
BATCH_SIZE = 24
EPOCHS = 100
LEARNING_RATE = 5e-5
WARM_STEPS = 200

models_path = drive_path + 'models/'

In [18]:
def train(
    dataset, output_prefix="MetalliA", save_model_on_epoch=True,
):
  gpt2.train()

  optimizer = AdamW(gpt2.parameters(), lr=LEARNING_RATE)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=WARM_STEPS, num_training_steps=-1
  )

  train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
  loss=0
  accumulating_batch_count = 0
  input_tensor = None
  min_loss = float("Inf")
  for epoch in range(EPOCHS):

    print(f"Training epoch {epoch}")
    
    for idx, entry in tqdm(enumerate(train_dataloader)):

      input_tensor = entry.to(device)
      outputs = gpt2(input_tensor, labels=input_tensor)
      loss = outputs[0]
      loss.backward()

      if (accumulating_batch_count % BATCH_SIZE) == 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        gpt2.zero_grad()

      accumulating_batch_count += 1
      
    if save_model_on_epoch:
        if loss < min_loss:
          torch.save(
              gpt2.state_dict(),
              os.path.join(models_path, f"{output_prefix}-{epoch}.pt"),
          )
          min_loss = loss
    print(loss)
    print()
        
  return gpt2

In [19]:
device=torch.device("cuda")
if os.path.isfile(drive_path+'models/MetalliA-best.pt'):
  gpt2_finetune = gpt2.cuda()
  gpt2_finetune.load_state_dict(torch.load(drive_path+'models/MetalliA-best.pt'))
  gpt2_finetune.eval()
else:
  gpt2 = gpt2.cuda()
  gpt2_finetune = train(tr_dataset)

In [20]:
def generate(input, max_len = 30, top_p = 0.8, temp=1.0):
  #model_cpu = model.to('cpu')
  gpt2_finetune.eval()

  device = torch.device("cuda")
  filter_value = -float("Inf")

  new_only = torch.tensor([]).unsqueeze(0).to(device)

  with torch.no_grad():
    generation = torch.tensor(tokenizer.encode(input)).unsqueeze(0).to(device)

    for i in tqdm(range(max_len)):
      outputs = gpt2_finetune(generation, labels = generation)
      loss, logits = outputs[:2]
      logits = logits[:, -1, :] / (temp if temp > 0 else 1.0)

      sorted_logits, sorted_index = torch.sort(logits, descending=True)
      cum_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim = -1)

      sorted_indices_to_remove = cum_probs > top_p
      sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
          ..., :-1
      ].clone()
      sorted_indices_to_remove[..., 0] = 0

      indices_to_remove = sorted_index[sorted_indices_to_remove]
      logits[:, indices_to_remove] = filter_value

      next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)

      if next_token in tokenizer.encode("<|endoftext|>"):
        break 

      new_only = torch.cat((new_only, next_token), dim=1)
      generation = torch.cat((generation, next_token), dim=1)
    
    output_list = list(generation.squeeze().cpu().numpy())
    output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 

    new_only_cpu = new_only.squeeze().cpu()

    if len(new_only_cpu.size()) > 0:
      only_gen_list = list(new_only_cpu.numpy())
      only_gen = f"{tokenizer.decode(only_gen_list)}<|endoftext|>"
    else:
      only_gen_list = []
      only_gen = f"<|endoftext|>"

  return output_text, only_gen

In [21]:
def text_generation(test_data):
  generated_lyrics = []
  full_lyrics = []
  for i in range(len(test_data)):
    full_song, gen_verses = generate(test_data['lyrics'][i])
    generated_lyrics.append(gen_verses)
    full_lyrics.append(full_song)
  return generated_lyrics, full_lyrics

In [22]:
generated_lyrics, full_lyrics = text_generation(test_set)

100%|██████████| 30/30 [00:01<00:00, 24.31it/s]


In [23]:
generated_lyrics

[" here before? <|endofverse|> No there ain't no heroes here. <|endofverse|> Haven't I seen you<|endoftext|>",
 " you. <|endofverse|> Now that I'm out of my head, <|endofverse|> The crumbling wreckage of your<|endoftext|>",
 ' on it rides <|endofverse|> Into the abyss of desire <|endofverse|> Into the intoxicating light <|end<|endoftext|>',
 " <|endofverse|> I'm trapped under ice <|endofverse|> Crystallized, as I lay here and rest <<|endoftext|>",
 ' Bound by the world all alone <|endofverse|> Crushed under heavy skies <|endofverse|> Crushed under heavy skies<|endoftext|>',
 ' Luck. Runs. Out. <|endofverse|> Luck. Runs. Out. <|endofverse|> There is something wrong<|endoftext|>',
 ' <|endofverse|> There are fingers in me <|endofverse|> Needles in my fingers <|endofverse|<|endoftext|>',
 ' <|endofverse|> Through the never <|endofverse|> Open door, never closed, for I know <|endof<|endoftext|>',
 ' suck the day <|endofverse|> Come carpe diem, baby <|endofverse|> Yeah, Suck it!<|endoftext|

In [24]:
test_set['Generated_lyrics'] = generated_lyrics

In [25]:
test_set.head(10)

,index,album,song_name,lyrics,true_lyrics,Generated_lyrics
0,94,reload,slither,"""Slither"" <|endofverse|> Don't go looking for ...",here before? <|endofverse|> Have your heroes d...,here before? <|endofverse|> No there ain't no...
1,83,black_album,the unforgiven,"""The Unforgiven"" <|endofverse|> New blood join...",you. <|endofverse|> So I dub thee unforgiven. ...,you. <|endofverse|> Now that I'm out of my he...
2,100,reload,better than you,"""Better Than You"" <|endofverse|> I look at you...","on and on <|endofverse|> Forever on and on, on...",on it rides <|endofverse|> Into the abyss of ...
3,2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno...",<|endofverse|> Can't hear my call <|endofverse...,<|endofverse|> I'm trapped under ice <|endofv...
4,37,hardwired_to_self_destruct,atlas rise,"""Atlas, Rise!"" <|endofverse|> Bitterness and b...",Bound by the world all alone <|endofverse|> Cr...,Bound by the world all alone <|endofverse|> C...
5,74,death_magnetic,all nightmare long,"""All Nightmare Long"" <|endofverse|> Luck. Runs...",Luck. Runs. Out. <|endofverse|> You crawl back...,Luck. Runs. Out. <|endofverse|> Luck. Runs. O...
6,98,reload,fixxxer,"""Fixxxer"" <|endofverse|> Dolls of voodoo all s...","<|endofverse|> No more, no more pins in me <|e...",<|endofverse|> There are fingers in me <|endo...
7,84,black_album,through the never,"""Through The Never"" <|endofverse|> All that is...",<|endofverse|> Turning <|endofverse|> Who we a...,<|endofverse|> Through the never <|endofverse...
8,102,reload,carpe diem baby,"""Carpe Diem Baby"" <|endofverse|> Hit dirt <|en...",suck the day <|endofverse|> Come make me miss ...,"suck the day <|endofverse|> Come carpe diem, ..."
9,76,death_magnetic,the day that never comes,"""The Day That Never Comes"" <|endofverse|> Born...",I swear <|endofverse|> The sun will shine <|en...,this is the end <|endofverse|> All alone in t...


In [26]:
for i in range(len(test_set)):
  print("song name:{}".format(test_set['song_name'][i]))
  print("TRUE lyrics:\t\t{}".format(test_set['true_lyrics'][i]))
  print("Model generation:\t{}".format(test_set['Generated_lyrics'][i]))
  
  print()

song name:slither
TRUE lyrics:		here before? <|endofverse|> Have your heroes disappeared? <|endofverse|> Don't send your eyes to the sun you might blind them... <|endofverse|>
Model generation:	 here before? <|endofverse|> No there ain't no heroes here. <|endofverse|> Haven't I seen you<|endoftext|>

song name:the unforgiven
TRUE lyrics:		you. <|endofverse|> So I dub thee unforgiven. <|endofverse|> Never free. <|endofverse|> Never me. <|endofverse|> So I dub thee unforgiven. <|endofverse|>
Model generation:	 you. <|endofverse|> Now that I'm out of my head, <|endofverse|> The crumbling wreckage of your<|endoftext|>

song name:better than you
TRUE lyrics:		on and on <|endofverse|> Forever on and on, on <|endofverse|> Better Than You <|endofverse|> Yeah <|endofverse|> Better Than You <|endofverse|>
Model generation:	 on it rides <|endofverse|> Into the abyss of desire <|endofverse|> Into the intoxicating light <|end<|endoftext|>

song name:trapped under ice
TRUE lyrics:		<|endofverse|> Ca

In [27]:
title = '"The sound of silence" <|endofverse|>'
full_song, _ = generate(title, max_len=350, temp = 1.15)

 93%|█████████▎| 326/350 [00:08<00:00, 37.41it/s]


In [28]:
print(full_song.replace('<|endofverse|>', '\n'))


"The sound of silence" 
 War is coming, no end 
 The sun is shining clean through, no need to hide 
 War is coming, war is here 
 Please, please send the message 
 Of no return 
 War is coming, it's never ending 
 Obey, obey, obey, obey 
 When you feel fear or sorrow, turn your eyes 
 You are no longer there, but the fear inside 
 War is coming 
 Pure gold reigns supreme 
 Pure power whispers within 
 They send darkness, but are not fear 
 Dark times are upon us all 
 Obey, obey, obey, obey 
 When you feel fear or sorrow, turn your eyes 
 You are no longer there, but the fear inside 
 Obey, obey, obey, obey 
 When you feel fear or sorrow, turn your eyes 
 You are no longer there, but the fear inside 
 Orking the shadows, but become so strong 
 Men of iron rule 
 Don't change your mind or look in<|endoftext|>
